In [3]:
import pandas as pd
import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import os
pi =  3.14159265
import math

In [2]:
dblist =os.listdir('./Data/db_label')

In [20]:
dblist

['F01-20211110T024918Z-001',
 'F02-20211110T024927Z-001',
 'F03-20211110T024929Z-001',
 'F04-20211110T024931Z-001',
 'F05-20211110T024933Z-001',
 'F06-20211110T024936Z-001',
 'F07-20211110T024938Z-001']

In [30]:
labelcsv

['Rec_154920.db.csv',
 'Rec_154944.db.csv',
 'Rec_155034.db.csv',
 'Rec_155134.db.csv',
 'Rec_155202.db.csv',
 'Rec_155228.db.csv',
 'Rec_155317.db.csv',
 'Rec_155348.db.csv',
 'Rec_155418.db.csv',
 'Rec_155547.db.csv',
 'Rec_155615.db.csv',
 'Rec_155640.db.csv']

In [3]:
df2 = pd.DataFrame(index=range(0,0))
for i in dblist:
    labelcsv = os.listdir(f'./Data/db_label/{i}')
    for j in labelcsv:
        df = pd.DataFrame(pd.read_csv(f"./Data/db_label/{i}/{j}"))
        df2 = pd.concat([df2,getDF(df)])
df2.dropna(axis=0, inplace = True)

### 상보필터

In [4]:
def getDF(df):
    df.a1 =  df.a1.astype('float64')
    df.a2 =  df.a2.astype('float64')
    df.a3 =  df.a3.astype('float64')
    df.g1 =  df.g1.astype('float64')
    df.g2 =  df.g2.astype('float64')
    df.g3 =  df.g3.astype('float64')
    df["gravity"] = (df.a1**2 + df.a2**2 + df.a3**2)**0.5

    df["accXAngle"] = 0.0
    df["comFX"]=0.0
    df["accYAngle"] = 0.0
    df["comFY"] = 0.0

    comFX = 0
    comFY = 0
    for i in range(len(df)):
        df["accXAngle"][i] =-(180/pi *math.atan(df.a3[i]/((df.a1[i]**2+df.a2[i]**2)**0.5)))
        df["accYAngle"][i]=(180/pi *math.atan(-df.a1[i]/((df.a3[i]**2+df.a2[i]**2)**0.5)))
        comFX = (0.95*(comFX+(df.g1[i]/131*0.000244140625)))+(0.05*df["accXAngle"][i])
        comFY = (0.95*(comFY+(df.g3[i]/131*0.000244140625)))+(0.05*df["accYAngle"][i])
        df["comFX"][i] = comFX
        df["comFY"][i] = comFY
    
#     df.drop(['ID','TIMESTAMP','seq','a1','a2', 'a3','g1','g2','g3','accXAngle','accYAngle'],axis = 1, inplace = True)
    return getlabel(df)

In [94]:
df2

,ID,TIMESTAMP,seq,a1,a2,a3,g1,g2,g3,result,gravity,accXAngle,comFX,accYAngle,comFY
0,1,155640.130,166,2.0,-82.0,2.0,0.0,1.0,0.0,0,82.048766,-1.396766,-0.069838,-1.396766,-0.069838
1,2,155640.142,167,2.0,-82.0,2.0,0.0,0.0,0.0,0,82.048766,-1.396766,-0.136185,-1.396766,-0.136185
2,3,155640.152,168,2.0,-82.0,1.0,0.0,0.0,1.0,0,82.030482,-0.698487,-0.164300,-1.397077,-0.199228
3,4,155640.161,169,2.0,-83.0,1.0,0.0,0.0,1.0,0,83.030115,-0.690077,-0.190589,-1.380254,-0.258277
4,5,155640.167,170,1.0,-83.0,1.0,0.0,0.0,1.0,0,83.012047,-0.690227,-0.215571,-0.690227,-0.279873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,712,155648.345,220,233.0,-5.0,-29.0,-82.0,21.0,-145.0,1,234.851017,7.093133,0.055702,-82.801662,-36.180441
712,713,155648.355,221,192.0,22.0,-15.0,-104.0,28.0,-138.0,1,193.837561,4.438236,0.274644,-82.104435,-38.476885
713,714,155648.365,222,136.0,48.0,-5.0,-103.0,48.0,-114.0,1,144.308697,1.985579,0.360009,-70.462728,-40.076379
714,715,155648.375,223,119.0,65.0,-5.0,-75.0,50.0,-94.0,1,135.687140,2.111798,0.447465,-61.284656,-41.136959


In [126]:
df2.to_csv('F01_07.csv',index=False)

### 30개씩 데이터 결합

In [71]:
def getlabel(df):
    y_value = []
    for i in range(1,31):
        globals()[f'gravity{i}']=[]
        globals()[f'comFX{i}']=[]   
        globals()[f'comFY{i}']=[]      
    for i in range(len(df)-30):
        check = 0
        for j in range(1,31):
            globals()[f'gravity{j}'].append(df.gravity[i+j])
            globals()[f'comFX{j}'].append(df.comFX[i+j])   
            globals()[f'comFY{j}'].append(df.comFY[i+j])
            if df.result[i+j]==1:
                check+=1
        if check > 15:
            y_value.append(1)
        else:
            y_value.append(0)
    a=dict(zip([f'gravity{i}' for i in range(1,31)],(globals()[f'gravity{i}'] for i in range(1,31)) ))
    a1=dict(zip([f'comFX{i}' for i in range(1,31)],(globals()[f'comFX{i}'] for i in range(1,31)) ))
    a2=dict(zip([f'comFY{i}' for i in range(1,31)],(globals()[f'comFY{i}'] for i in range(1,31)) ))
    a3 = {'y_value': y_value}
    a.update(a1)
    a.update(a2)
    a.update(a3)  
    
    return pd.DataFrame(a)

In [9]:
df2.drop(['ID','TIMESTAMP','seq','a1','a2', 'a3','g1','g2','g3','accXAngle','accYAngle'],axis = 1, inplace = True)

In [35]:
aa = getlabel(df2)
aa

,gravity1,gravity2,gravity3,gravity4,gravity5,gravity6,gravity7,gravity8,gravity9,gravity10,...,comFY22,comFY23,comFY24,comFY25,comFY26,comFY27,comFY28,comFY29,comFY30,y_value
0,91.000000,81.344945,74.632433,72.062473,71.133677,71.596089,72.470684,73.437048,73.437048,72.367120,...,-4.319959,-4.492379,-4.669289,-4.831402,-5.101335,-5.369326,-5.620742,-5.817350,-5.878345,0
1,81.344945,74.632433,72.062473,71.133677,71.596089,72.470684,73.437048,73.437048,72.367120,72.367120,...,-4.492379,-4.669289,-4.831402,-5.101335,-5.369326,-5.620742,-5.817350,-5.878345,-5.890581,0
2,74.632433,72.062473,71.133677,71.596089,72.470684,73.437048,73.437048,72.367120,72.367120,72.443081,...,-4.669289,-4.831402,-5.101335,-5.369326,-5.620742,-5.817350,-5.878345,-5.890581,-5.894293,0
3,72.062473,71.133677,71.596089,72.470684,73.437048,73.437048,72.367120,72.367120,72.443081,72.842295,...,-4.831402,-5.101335,-5.369326,-5.620742,-5.817350,-5.878345,-5.890581,-5.894293,-5.828447,0
4,71.133677,71.596089,72.470684,73.437048,73.437048,72.367120,72.367120,72.443081,72.842295,73.273460,...,-5.101335,-5.369326,-5.620742,-5.817350,-5.878345,-5.890581,-5.894293,-5.828447,-5.701035,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,85.912746,84.083292,82.468176,82.225300,82.024387,79.856121,78.854296,77.672389,77.768888,76.131465,...,0.812604,0.771962,0.733349,0.735208,0.698436,0.663503,0.525584,0.329402,0.015814,1
271,84.083292,82.468176,82.225300,82.024387,79.856121,78.854296,77.672389,77.768888,76.131465,75.398939,...,0.771962,0.733349,0.735208,0.698436,0.663503,0.525584,0.329402,0.015814,-0.307085,1
272,82.468176,82.225300,82.024387,79.856121,78.854296,77.672389,77.768888,76.131465,75.398939,74.946648,...,0.733349,0.735208,0.698436,0.663503,0.525584,0.329402,0.015814,-0.307085,-0.669176,1
273,82.225300,82.024387,79.856121,78.854296,77.672389,77.768888,76.131465,75.398939,74.946648,75.973680,...,0.735208,0.698436,0.663503,0.525584,0.329402,0.015814,-0.307085,-0.669176,-0.979339,1
